In [1]:
from __future__ import print_function
import matplotlib
matplotlib.use('Qt5Agg')

from pylab import *
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import networkx as nx
import random as rd

height=36
width=1000
t_count=0
threshold = 2

def init(gi):
    global g, positions, s, position_2, label
    if gi == 3:
        g = nx.navigable_small_world_graph(5, p=1, q=1, r=2, dim=2, seed=None)
        label = 'Small world network'
        positions = nx.spring_layout(g)
    elif gi == 2:
        g=nx.random_regular_graph(3, 20, seed=None)
        label = 'Random lattice'
        positions = nx.circular_layout(g)
    elif gi == 5:
        g=nx.barabasi_albert_graph(25, 4, seed=None)
        label = 'Barabasi Albert graph'
        positions = nx.circular_layout(g)
    elif gi == 4:
        g=nx.fast_gnp_random_graph(15,1, seed=None, directed=False)
        label = 'Erdos Renyi network'
        positions = nx.circular_layout(g)
    elif gi == 1:
        g=nx.random_powerlaw_tree(20, 3, 0)
        label = 'Random powerlaw graph'
        positions = nx.spring_layout(g)
    else:
        g = nx.karate_club_graph()
        label = 'Karate Club'
        positions = nx.spring_layout(g)
        
    for i in g:
        if random() < .5: 
            g.node[i]['pol'] = 1
        else:
            g.node[i]['pol'] = -1
        if random() < .5: 
            g.node[i]['eco'] = 1
        else:
            g.node[i]['eco'] = -1
    #positions = nx.kamada_kawai_layout(g)
    
def update():
    global g, labels
    while True:
        speaker = rd.choice(list(g.nodes()))
        if not list(g.neighbors(speaker)):
            continue
        listener = rd.choice(list(g.neighbors(speaker)))
        break
    if random() < .5:
        update_pol(speaker, listener)
    else: 
        update_eco(speaker, listener)
    to_remove = []
    # listing the stubborn nodes that delete connections
    for p1 in g.nodes:
        pol1, eco1 = g.node[p1]['pol'], g.node[p1]['eco']
        for p2 in g.neighbors(p1):
            pol2, eco2 = g.node[p2]['pol'], g.node[p2]['eco']
            if abs(pol1) == threshold and abs(pol2) == threshold:
                if pol1*pol2<0 or eco1*eco2<0:
                    if (p2, p1) not in to_remove:
                        to_remove.append((p1, p2))
    for pair in list(set(to_remove)):
        g.remove_edge(*pair)            
    labels={}
    for i in g:
        labels[i]= str(int(g.node[i]['pol']))+","+str(int(g.node[i]['eco']))
    return speaker, listener

def update_pol(speaker, listener):
    global g
    l_pol = g.node[listener]['pol']
    s_pol = g.node[speaker]['pol']
    if abs( l_pol ) < threshold:
        if s_pol != 0:
            g.node[listener]['pol'] = l_pol + (s_pol)/abs(s_pol)
        elif l_pol != 0:
            g.node[listener]['pol'] = l_pol - (l_pol)/abs(l_pol)
        else:
            pass

def update_eco(speaker, listener):
    ## update food
    global g 
    l_eco = g.node[listener]['eco']
    s_eco = g.node[speaker]['eco']
    if abs( l_eco ) < threshold:
        if s_eco != 0:
            g.node[listener]['eco'] = l_eco + (s_eco)/abs(s_eco)
        elif l_eco != 0:
            g.node[listener]['eco'] = l_eco - (l_eco)/abs(l_eco)
        else:
            pass

def init_fig():
    global fig, viz_pol, viz_eco, ax1, ax2, g, viz2
    fig = plt.figure(figsize=(15,7))
    ax1 = plt.subplot(1,2,1)
    viz_pol = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 800, edge_color='w',
                  node_color = [choose_pol_color(g.nodes[n]['pol']) for n in g.nodes])
    viz_eco = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 250, edgelist = [],
                  node_color = [choose_eco_color(g.nodes[n]['eco']) for n in g.nodes])
    fig.set_facecolor("#000000")
    ax2 = plt.subplot(1,2,2)
    plt.axis([0, width, 0, height])
    plt.suptitle(label)
    plt.xlabel('Time', fontsize=30)
    plt.ylabel('Node Count', fontsize=30)

def choose_pol_color(pol):
    val = (pol+threshold)/2/threshold
    return plt.cm.seismic(val)

def choose_eco_color(eco):
    val = (eco+threshold)/2/threshold
    return plt.cm.seismic(val)

    
def observe1(ax, s=None, l=None):
    global g, labels, viz_pol, viz_eco
    cla()
    def get_edge_color(edge, s, l):
        if edge == (s, l) or edge == (l, s):
            return 'r'
        else:
            return 'k'
    if s and l:
        edge_colors = [get_edge_color(edge, s, l) for edge in g.edges]
    else:
        edge_colors = 'k'
    plt.sca(ax)    
    viz_pol = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 800, edge_color=edge_colors,
                  node_color = [choose_pol_color(g.nodes[n]['pol']) for n in g.nodes])
    viz_eco = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 250, edgelist=[],
                  node_color = [choose_eco_color(g.nodes[n]['eco']) for n in g.nodes])
    nx.draw_networkx_labels(g,positions,labels,#font_weight='bold',
                            font_color=plt.cm.viridis(0.2), font_size=18)
    #fig = plt.gcf()
    fig.set_facecolor(plt.cm.Greys(0.3))
    return fig

def observe2(ax, xplot, yplot_pp, yplot_mp,yplot_mm,yplot_pm,yplot_oo):
    global g, viz2,t,width,height,t_count,labels
    count1, count2, count3,count4,count5 = 0, 0, 0, 0, 0
    
    for i in g: 
        if (g.node[i]["pol"]>=1 and g.node[i]["eco"]>=0) or (g.node[i]["pol"]>=0 and g.node[i]["eco"]>=1):
            count1+=1
        elif (g.node[i]["pol"]<=0 and g.node[i]["eco"]>=1) or (g.node[i]["pol"]<=-1 and g.node[i]["eco"]>=0):
            count2+=1
        elif (g.node[i]["pol"]<=0 and g.node[i]["eco"]<=-1) or (g.node[i]["pol"]<=-1 and g.node[i]["eco"]<=0):
            count3+=1
        elif (g.node[i]["pol"]>=0 and g.node[i]["eco"]<=-1) or (g.node[i]["pol"]>=1 and g.node[i]["eco"]<=0):
            count4+=1 
        elif g.node[i]["pol"]==0 and g.node[i]["eco"]==0:
            count5+=1
    #print("0: " + str(count0))
    #print("1: " + str(count1))
    plt.hold(True)
    xplot.append(t_count)
    yplot_pp.append(count1)
    yplot_mp.append(count2)
    yplot_mm.append(count3)
    yplot_pm.append(count4)
    yplot_oo.append(count5)
    ax.plot(xplot,yplot_pp,'r')
    ax.plot(xplot,yplot_mp,'b')
    ax.plot(xplot,yplot_mm,'k')
    ax.plot(xplot,yplot_pm,'g')
    ax.plot(xplot,yplot_oo,'y')
    ax.set_xlabel('Time', fontsize=30)
    ax.set_ylabel('Node Count', fontsize=30)
    #plt.legend()
    ax.set_xlim(0, max(xplot)+1)
    ax.set_ylim(ymin=0, auto=True)
    plt.suptitle(label,fontsize=30)
    return xplot, yplot_pp, yplot_mp,yplot_mm,yplot_pm,yplot_oo
    
prompt_text = """
Enter graph index...
1. Random Powerlaw Tree
2. Lattice
3. Small World Network
4. Erdos Renyi Network
5. Barabasi Albert Graph
Default: Karate Club Graph
\t"""

try:
    graph_type = int(input(prompt_text))
except:
    graph_type = 0
    
init(graph_type)
init_fig()
step = 5000

def step(t):
    global t_count, xplot, yplot_pp, yplot_mp,yplot_mm,yplot_pm,yplot_oo
    if t == 0:
        xplot, yplot_pp, yplot_mp,yplot_mm,yplot_pm,yplot_oo= [], [], [],[],[],[]
    t_count=t_count+1
    xplot, yplot_pp, yplot_mp, yplot_mm, yplot_pm, yplot_oo=observe2(ax2, xplot, yplot_pp, yplot_mp,yplot_mm,yplot_pm,yplot_oo)
    s, l = update()
    observe1(ax1, s, l)
    return  viz_pol, viz_eco, viz2,fig

ani = FuncAnimation(fig, step, interval=0)
show()


Enter graph index...
1. Random Powerlaw Tree
2. Lattice
3. Small World Network
4. Erdos Renyi Network
5. Barabasi Albert Graph
Default: Karate Club Graph
	1


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:176: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\__init__.py:911: MatplotlibDeprecationWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  mplDeprecation)
C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\rcsetup.py:156: MatplotlibDeprecationWarning: axes.hold is deprecated, will be removed in 3.0
  mplDeprecation)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\cbook\__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\cbook\__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
  File "C:\ProgramData\Anaconda2\lib\site-packages\ma